In [1]:
!pip install transformers
!pip install datasets
!pip install fugashi
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 78.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=eda5b627954996f7aed71fd13e203f3824226b0078a5f9491eb44dc87f44967f
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [2]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ja")

train_dataset = dataset["train"]
train_dataset

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6505it [00:00, 45546.80it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1146it [00:00, 47065.64it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2464it [00:00, 70190.88it/s]


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 6505
})

In [3]:
# 選擇特定的行和列
num_row = 10
data = train_dataset.select(range(num_row))
selected_data = data.from_dict({
    'audio': data['audio'],
    'sentence': data['sentence']
})
selected_data

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 10
})

In [4]:
import torchaudio

def process_waveforms(batch):

  waveform, sample_rate = torchaudio.load(batch['audio']['path'])

  if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

  # 如果 waveform 是雙聲道，需要轉單聲道。給 4GE用
  if waveform.size(0) > 1:
    waveform = waveform.mean(dim=0)

  # 讓 waveform的維度正確
  if waveform.ndim > 1:
    waveform = waveform.squeeze()

  batch["speech_array"] = waveform
  batch["sample_rate"] = 16000

  batch['labels'] = batch['sentence']

  return batch

selected_data = selected_data.map(process_waveforms, remove_columns=selected_data.column_names)

selected_data

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['speech_array', 'sample_rate', 'labels'],
    num_rows: 10
})

In [5]:
from transformers import AutoProcessor, AutoTokenizer

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-char")
blank_token = "[BLANK]"

if blank_token not in tokenizer.get_vocab():
  tokenizer.add_tokens([blank_token])

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/110 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

# 動態 dataset，對於每個 batch的 max_length做填補

In [6]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

batch_size = 2

def dynamic_padding_collate_fn(batch):
  
  X, y, input_lengths, target_lengths = zip(*batch)

  # aduio padding
  padded_X = pad_sequence(X, batch_first=True, padding_value=0.0)

  # labels padding
  padded_y = pad_sequence(y, batch_first=True, padding_value=tokenizer.pad_token_id)

  return padded_X, padded_y, torch.tensor(input_lengths), torch.tensor(target_lengths)

def make_dataset(data, processor, tokenizer, batch_size):
  X = []
  y = []
  input_lengths = []
  target_lengths = []

  for i in range(len(data['speech_array'])):
    # 單獨處理
    processed_audio = processor(data['speech_array'][i],
                   sampling_rate=16000,
                   return_tensors="pt",
                   padding=False,
                   truncation=True,
                   max_length=160000)

    
    tokenized_label = tokenizer(data['labels'][i],
                   return_tensors="pt",
                   padding=False,
                   truncation=True,
                   max_length=32)

    X.append(processed_audio.input_values.squeeze(0))
    y.append(tokenized_label.input_ids.squeeze(0))
    input_lengths.append(processed_audio.attention_mask.sum())
    target_lengths.append(tokenized_label.attention_mask.sum())

  dataset = list(zip(X, y, input_lengths, target_lengths))
  loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, collate_fn=dynamic_padding_collate_fn)

  return loader

loader = make_dataset(selected_data, processor, tokenizer, batch_size)

# 靜態 dataset，用整個 dataset的 max_length做填補

In [7]:
# from torch.utils.data import DataLoader

# batch_size = 2

# def make_dataset(data, processor, tokenizer, batch_size):
#   processed_audios = processor(data['speech_array'],
#           sampling_rate=16000,
#           return_tensors="pt",
#           padding=True,
#           truncation=True,
#           max_length=160000)
#   tokenized_labels = tokenizer(data['labels'],
#           return_tensors="pt",
#           padding='max_length',
#           truncation=True,
#           max_length=32)

#   X = processed_audios.input_values
#   y = tokenized_labels.input_ids

#   input_lengths = processed_audios.attention_mask.sum(dim=1)
#   target_lengths = tokenized_labels.attention_mask.sum(dim=1)

#   dataset = list(zip(X, y, input_lengths, target_lengths))
#   loader = DataLoader(dataset, shuffle=True, batch_size=batch_size)

#   return loader

# loader = make_dataset(selected_data, processor, tokenizer, batch_size)

In [8]:
import torch

for X_batch, y_batch, input_lengths, target_lengths in loader:
  print(X_batch.size())
  print(y_batch.size())
  print(tokenizer.decode(y_batch[0]))
  print(y_batch)
  print(input_lengths)
  print(target_lengths)
  print('===================')

torch.Size([2, 57600])
torch.Size([2, 14])
[CLS] 当 番 を 決 め て 終 わ り で す 。 [SEP]
tensor([[  2, 191, 320,  15, 368,  68,  18, 394, 106,  39,  17,  46,  10,   3],
        [  2, 439,   6, 530,  15, 313, 148,  32,  79,  31,  23,   9,  10,   3]])
tensor([50688, 57600])
tensor([14, 14])
torch.Size([2, 66240])
torch.Size([2, 25])
[CLS] 待 っ て ろ よ 、 馬 鹿 ど も 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor([[   2,  980,   30,   18,  390,   54,    7,  486, 1151,   98,   42,   10,
            3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   2,   85,   49,  119,   13,   72,   37,   23,   18,   11,    6,   15,
          236,   11,  127,   79,   17,  343,  313,  148,   54,   66,   24,   11,
            3]])
tensor([44928, 66240])
tensor([13, 25])
torch.Size([2, 53568])
torch.Size([2, 22])
[CLS] お い お い 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor([[   2,   87

# 用 model訓練再跟 label做 CTC loss

In [9]:
import torch
import torch.nn as nn
from transformers import HubertModel

class CustomModelWithHubert(nn.Module):
    def __init__(self, num_classes):
        super(CustomModelWithHubert, self).__init__()
        self.hubert = HubertModel.from_pretrained("rinna/japanese-hubert-base")

        self.conv1d = nn.Conv1d(in_channels=768, out_channels=512, kernel_size=3, padding=1, stride=1)
        self.leaky_relu = nn.LeakyReLU()
        self.fc = nn.Linear(512, num_classes)  # 全連接層，將輸出特徵映射到字母序列

    def forward(self, inputs, attention_mask=None):
        # 使用Hubert模型提取特徵
        outputs = self.hubert(inputs, attention_mask=attention_mask) # -> [batch_size, sequence_length, 768 features]
        x = outputs.last_hidden_state.transpose(1, 2) # -> [batch_size, 768 features, sequence_length]
        print('last_hidden_state ',x)

        # 使用卷積層進行特徵處理
        x = self.conv1d(x) # -> [batch_size, 512 features, sequence_length]
        print('conv1d ',x)

        x = self.leaky_relu(x)
        print('leaky_relu ',x)

        batch_size, channels, sequence_length = x.size() # = [1, 512, 233]
        print('batch_size, channels, sequence_length ',x.size())
        x = x.view(batch_size * sequence_length, channels)

        # 使用全連接層映射到字母序列
        x = self.fc(x) # -> [batch_size * sequence_length, num_classes]
        print('fc ',x)

        # 恢復時步
        x = x.view(batch_size, sequence_length, -1) # -> [batch_size, sequence_length, num_classes]

        sequence_length,_ ,_ = x.size()

        return x, sequence_length

NUM_TOKENS = 4001

model = CustomModelWithHubert(num_classes=NUM_TOKENS)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at rinna/japanese-hubert-base were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at rinna/japanese-hubert-base and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [10]:
wav_input_16khz = torch.randn(4, 80000)
outputs,_ = model(wav_input_16khz)
print(f"Input:   {wav_input_16khz.size()}")  # [1, 10000]
print(f"Output:  {outputs.size()}")  # [1 batch_size, 31 sequence_length, 768 features]


last_hidden_state  tensor([[[ 0.0190, -0.0283, -0.0285,  ..., -0.1071, -0.0939, -0.0841],
         [-0.0521, -0.1101, -0.1180,  ...,  0.0587,  0.0532, -0.0242],
         [-0.0041, -0.0357,  0.0525,  ...,  0.1239,  0.1976,  0.1294],
         ...,
         [-0.0983, -0.1007, -0.0940,  ..., -0.1307, -0.1333, -0.1260],
         [ 0.1092,  0.0807,  0.0296,  ..., -0.0298, -0.0729, -0.0897],
         [-0.6836, -0.9915, -0.5247,  ..., -1.0122, -0.5541, -0.4753]],

        [[ 0.0041, -0.0397, -0.0103,  ..., -0.1110, -0.1493, -0.1305],
         [-0.0280, -0.0732, -0.0873,  ..., -0.0058,  0.0033, -0.0362],
         [-0.0491, -0.1186,  0.0816,  ...,  0.1196,  0.1112,  0.0790],
         ...,
         [-0.0880, -0.0995, -0.1042,  ..., -0.1444, -0.1389, -0.1062],
         [ 0.1045,  0.0674,  0.0821,  ..., -0.0404, -0.0842, -0.1173],
         [-0.4765, -0.4413, -1.3482,  ..., -1.1701, -1.1673, -0.4817]],

        [[ 0.0320, -0.0139, -0.0208,  ..., -0.1179, -0.1308, -0.1599],
         [-0.0535, -0.1456

In [11]:
import torch

class Trainer:
  def __init__(self, model, optimizer, loss_fn):
    self.model = model
    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.total_loss = 0
    self.model.eval() # 不打這行，model也本來就預設evaluate mode

  def training_one_batch(self, X_batch, y_batch, input_lengths, target_lengths):
    self.model.train()

    # forward and predict & calculate CTC_loss

    y_pred, new_length = self.model(X_batch)

   # print('y_pred: ',y_pred.size())
   # print('y_batch: ',y_batch.size())

    log_probs = torch.nn.functional.log_softmax(y_pred, dim=2)
    print('log_probs ',log_probs)

    # [batch_size, sequence_length, num_classes] -> [sequence_length, batch_size, num_classes]
    log_probs = log_probs.transpose(0, 1)

   # new_lengths = [233, 233, 233]
    new_lengths = [new_length for i in range(batch_size)]
    new_lengths = torch.tensor(new_lengths)

    loss = self.loss_fn(log_probs, y_batch, new_lengths, target_lengths) # input_lengths要 [233, XXX]
   # print("loss: ",loss)
    self.total_loss += loss

    # Backward and optimize
    self.optimizer.zero_grad()
    loss.backward()

    # 梯度裁切
    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

    self.optimizer.step()

    return

  def evaluate(self, X_batch, y_batch, input_lengths, target_lengths):
    self.model.eval()

    with torch.no_grad():
      y_pred = self.model(X_batch)
    return y_pred
    # return self.loss_fn(y_pred, y_batch, input_lengths, target_lengths)

In [12]:
blank_token_index = tokenizer.convert_tokens_to_ids('[BLANK]')
tokenizer.convert_ids_to_tokens(4000)
print("token index:", blank_token_index)

token index: 4000


In [13]:
from torch import optim

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)
ctc_loss = nn.CTCLoss(blank=4000, zero_infinity=True, reduction='mean')

trainer = Trainer(model, optimizer, ctc_loss)

# Loss一開始就是0

In [14]:
# Training loop
num_epochs = 3

for epoch in range(num_epochs):
  trainer.total_loss = 0  # Reset total loss for each epoch
  num_batches = 0  # To count the number of batches processed

  for X_batch, y_batch, input_lengths, target_lengths in loader:
    trainer.training_one_batch(X_batch, y_batch, input_lengths, target_lengths)

    print('total loss: ', trainer.total_loss)
    num_batches = num_row // batch_size
    # If the last batch is smaller than the batch size, account for it
    if num_row % batch_size != 0:
        num_batches += 1

  average_loss = trainer.total_loss / num_batches
  print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")


last_hidden_state  tensor([[[ 0.1241,  0.0682,  0.0940,  ...,  0.1961,  0.0036, -0.0701],
         [-0.0453,  0.0025, -0.1886,  ..., -0.0078, -0.1883, -0.1307],
         [-0.2489, -0.2488, -0.2916,  ...,  0.0343, -0.1424, -0.1981],
         ...,
         [-0.0226, -0.1098, -0.0980,  ...,  0.1529, -0.0305, -0.0030],
         [ 0.0423, -0.0192, -0.1098,  ..., -0.1711, -0.0608, -0.1101],
         [-0.1102,  0.1088, -0.3266,  ..., -0.0431, -0.3154,  0.5278]],

        [[ 0.0398,  0.0749,  0.0628,  ..., -0.1274, -0.0750, -0.0889],
         [-0.1322, -0.0699,  0.0717,  ..., -0.1443,  0.0190, -0.0206],
         [-0.2142, -0.2504, -0.1877,  ..., -0.1828, -0.2426, -0.1799],
         ...,
         [-0.0621, -0.0716, -0.0853,  ...,  0.0560,  0.1300,  0.1116],
         [-0.0207,  0.0417,  0.0176,  ..., -0.0402,  0.0627,  0.0514],
         [ 0.1887, -0.0516, -0.4876,  ...,  0.1449,  0.0907,  0.1079]]],
       grad_fn=<TransposeBackward0>)
conv1d  tensor([[[ 0.1352,  0.4047,  0.2195,  ...,  0.0075, 

In [ ]:
for X_batch, y_batch, input_lengths, target_lengths in loader:
  print(trainer.evaluate(X_batch, y_batch, input_lengths, target_lengths))

last_hidden_state  tensor([[[ 0.0826,  0.0684,  0.0878,  ..., -0.0044,  0.0078,  0.0205],
         [-0.1105, -0.1138, -0.0606,  ..., -0.0836, -0.0753, -0.0705],
         [-0.2037, -0.0777, -0.0221,  ..., -0.2844, -0.2836, -0.3351],
         ...,
         [-0.0757, -0.0531, -0.0252,  ...,  0.0807,  0.0756,  0.0660],
         [ 0.0338, -0.0032, -0.0515,  ...,  0.0590,  0.0657,  0.0410],
         [ 0.0800, -0.0667, -0.4856,  ...,  0.0239,  0.0524,  0.1492]],

        [[ 0.1385,  0.1025,  0.0495,  ...,  0.1214, -0.0418,  0.0190],
         [-0.0641, -0.1566, -0.1403,  ...,  0.2053, -0.2228, -0.1446],
         [-0.2088, -0.1080, -0.0339,  ..., -0.0551, -0.2617, -0.3474],
         ...,
         [-0.0440, -0.0334, -0.0802,  ...,  0.0676,  0.0879,  0.0610],
         [ 0.0411,  0.0437, -0.0761,  ..., -0.1128,  0.0264,  0.0277],
         [ 0.0329, -0.0222, -0.3011,  ..., -0.1078, -0.0105,  0.3188]]])
conv1d  tensor([[[ 0.1809,  0.3729,  0.1944,  ...,  0.0081,  0.0457,  0.0193],
         [-0.0573,

In [ ]:
# # 日文的 ASR model(token = 32000)
# !pip install git+https://github.com/rinnakk/nue-asr.git

# import nue_asr
# tokenizer = nue_asr.load_tokenizer("rinna/nue-asr")
# num_tokens = len(tokenizer)